In [5]:
import json
import numpy as np
import pandas
#import os
#import io
from pprint import pprint
pandas.options.display.max_colwidth = -1

In [6]:
# runs in about 4m 30s
with open('events.json','r') as f:
    allgames = json.load(f)
print('ok')

ok


In [7]:
# runs in a few seconds
nEvents = 0
print('{0:,.0f} games found'.format(len(allgames)))
for game in allgames:
    nEvents = nEvents + len(game['events'])
df = pandas.DataFrame.from_dict(allgames)
df = df.set_index('id')
print('{0:,.0f} events found'.format(nEvents))

152,259 games found
14,946,418 events found


In [8]:
#firstyear = 2016
#lastyear = 2016
#theseyears = range(firstyear,lastyear+1)

column_list = ['event_in_game', 'event_type', 'inning', 'visitor_or_home']#,'retrosheet_id']
column_list += ['batter_id', 'pitch_count', 'pitch_list']
column_list += ['event', 'comment', 'sub_batting_order', 'sub_position']
column_list += ['gameID']

events_df = pandas.DataFrame(columns=column_list)

#for thisoneyear in theseyears:
#    print('Processing events for the year {0:.0f}...'.format(thisoneyear))
    
    
#print(len(df))    # 152259
dfx = df.iloc[0:10000]

i = 0
for index, thisgame in dfx.iterrows():
    for thisevent in thisgame['events']:
        if (np.mod(i,10000) == 0):
            print('Processing event {0:,.0f}...'.format(i))
        thisevent.append(index)
        events_df.loc[i] = thisevent
        i = i + 1

events_df['event_in_game'] = events_df['event_in_game'].apply(lambda x: int(x))
events_df = events_df.sort_values(['gameID','event_in_game'])
events_df.reset_index(drop=True)
#events_df = events_df.drop('index')
events_df.index.name = 'eventID'
events_df.to_csv('events1.csv')
print('Done')

Processing event 0...
Processing event 10,000...
Processing event 20,000...
Processing event 30,000...
Processing event 40,000...
Processing event 50,000...
Processing event 60,000...
Processing event 70,000...
Processing event 80,000...
Processing event 90,000...
Processing event 100,000...
Processing event 110,000...
Processing event 120,000...
Processing event 130,000...
Processing event 140,000...
Processing event 150,000...
Processing event 160,000...


KeyboardInterrupt: 

In [ ]:
# 100 appears at 4s
# 1,000 appears at 12s
# 10,000 appears at 2m 1s

In [ ]:
firstyear = 2016
lastyear = 2016
theseyears = range(firstyear,lastyear+1)

column_list = ['event_in_game', 'event_type', 'inning', 'visitor_or_home']#,'retrosheet_id']
column_list += ['batter_id', 'pitch_count', 'pitch_list']
column_list += ['event', 'comment', 'sub_batting_order', 'sub_position']
column_list += ['gameID']

events_df = pandas.DataFrame(columns=column_list)
i = 0
for thisoneyear in theseyears:
    print('Processing events for the year {0:.0f}...'.format(thisoneyear))
    dfx = df[pandas.to_datetime(df.index.str.slice(3,7)).year == thisoneyear]

    for index, row in dfx.iterrows():
        thisrow = []
        for j in range(0,len(row['events'])):
            thisrow = row['events'][j]
        if (len(thisrow) == 11):
            thisrow.append(index)
        events_df.loc[i] = thisrow
        i = i + 1

#filename = 'events' + str(firstyear) + '-' + str(lastyear) + '.csv'
#events_df.index.name = 'eventID'
#events_df.to_csv(filename)
#print('Saved to: '+filename)        
#events_df.shape
print('done')

In [ ]:
# for 1981-1990, runs in < 22 min and returns 20,337 events written to events1981-1990.csv
events_df_backup = events_df
#events_df = events_df_backup

In [ ]:
events_df = events_df_backup
events_df['event_in_game'] = events_df['event_in_game'].apply(lambda x: int(x))
events_df = events_df.sort_values('event_in_game')
#events_df = events_df.sort_values(['gameID', 'event_in_game'], ascending=[True, True])
events_df.head(10)
#events_df['event_in_game'].dtype


In [ ]:
#weird_game_ids = ['CLE192708190', 'BOS193008310', 'BOS193107250', 'CHA193304220', 'NYA193606030', 'MLN195505152']
#weird_game_ids += ['MIN196907170', 'NYN197208160', 'KCA201109150', 'TOR201208130', 'KCA201605310']

events_log = np.empty((0,12),dtype=object)

nGames = 0
for game in allgames:     # iterate through all games 
#    if (game['id'] in weird_game_ids):  # if the game is in the list above...
    nGames = nGames + 1
    if (np.mod(nGames,1000) == 0):
        print('Processing game {0:,.0f}...'.format(nGames))
    for j in range(0,len(game['events'])):         # iterate through each event in this game
        thisevent = np.empty((1,12),dtype=object)
        thisevent[0,0] = game['id']  # save game ID first
        for k in range(1,len(game['events'][j])):
            thisevent[0,k] = game['events'][j][k-1]  # then copy all rows from a single event
        events_log = np.vstack((events_log, thisevent))   # after each event processed, append to big list

print('\n')            
print('{0:,.0f} games processed'.format(nGames))

print('Loaded into array: {0:,.0f} events'.format(len(events_log)))


# we loaded everything into a numpy array, now save it as a pandas dataframe
events_df = pandas.DataFrame(events_log)

# pandas has auto-created an index variable
events_df.index.name = 'event_id'

column_list = ['gameID', 'event_in_game', 'event_type', 'inning', 'visitor_or_home', 'batter_id', 'pitch_count', 'pitch_list']
column_list += ['event', 'comment', 'sub_batting_order', 'sub_position']
events_df.columns = column_list
#events_df
print('Done')







In [ ]:

i = 0
while True:                   # proceed row by row through the events dataframe...
    if (i == len(events_df)):      # until the counter exceeds the length of the dataframe...
        break                        # then stop
    else:
        if (events_df['event_type'].loc[i] == 'com'):
            lastplay = i - 1
            events_df['comment'].loc[lastplay] += events_df['comment'].loc[i]
#            tester = tester.append(events_df.loc[i-1])
#            tester = tester.append(events_df.loc[i])
            while True:
#                events_df = events_df.drop(i)
                i = i + 1
                if (events_df['event_type'].loc[i] == 'com'):
                    events_df['comment'].loc[lastplay] += events_df['comment'].loc[i]
                else:
                    break
        else:
            i = i + 1

events_df = events_df[events_df['event_type'] != 'com']



In [ ]:
# renumber events in game
i = 0
i = i + 1
j = 1
while True:
#    if (np.mod(i,100) == 0):
#        print('Processing event number {0:,.0f}...'.format(i))
    if (i == len(events_df)):
        break
    else:
        events_df['event_in_game'].iloc[i] = j
        j = j + 1
        if (events_df['gameID'].iloc[i] != events_df['gameID'].iloc[i-1]):
            j = 1
    i = i + 1
    
# re-index eventID also
events_df = events_df.reset_index()

events_df[['gameID', 'event_in_game', 'event_type', 'event', 'comment']].iloc[75:95]#[events_df['gameID'] == 'CLE192708190']